In [ ]:
import pandas as pd

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

# Run FaIR

In [ ]:
baseline_scenario = "high"

In [ ]:
f = FAIR(ch4_method="Thornhill2021")

In [ ]:
f.define_time(1750, 2300, 1)  # start, end, step

In [ ]:
scenarios = [
    "baseline",
    "instant-removal-offsetting",
    "physical-delay-offsetting",
    "accelerated-emissions-offsetting",
    "upfront-embodied-offsetting",
]

In [ ]:
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_1_file = (
    "../data/original/calibrated_constrained_parameters_calibration1.4.1.csv"
)

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_1_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
fair_species_configs_1_4_1_file = (
    "../data/original/species_configs_properties_calibration1.4.1.csv"
)

In [ ]:
species, properties = read_properties(filename=fair_species_configs_1_4_1_file)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_csv(
    emissions_file="../data/modified/emissions_" + baseline_scenario + ".csv",
    forcing_file="../data/modified/volcanic.csv",
)

In [ ]:
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic")
    * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_1_file)
f.override_defaults(fair_params_1_4_1_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

# Save output

In [ ]:
f.temperature.to_netcdf(
    "../data/outputs/temperature_" + baseline_scenario + "_emissions.nc"
)

In [ ]:
pd.DataFrame(f.timebounds).to_csv("../data/outputs/timebounds.csv", index=False)